In [19]:
! pip install kagglehub

In [20]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("abinash6000/sle-gene-expression-profiles-7000")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/sle-gene-expression-profiles-7000


In [21]:
path = ' /root/.cache/kagglehub/datasets/abinash6000/sle-gene-expression-profiles-7000/versions/1'

In [22]:
import os

path = "/root/.cache/kagglehub/datasets/abinash6000/sle-gene-expression-profiles-7000/versions/1"

for root, dirs, files in os.walk(path):
    level = root.replace(path, "").count(os.sep)
    indent = " " * (4 * level)
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = " " * (4 * (level + 1))
    for f in files:
        print(f"{sub_indent}{f}")

1/
    Control/
        GSM1228916_6628_Control_SLE_G3_HGU133P.csv
        GSM1228861_9394_Control_SLE_G10_SPR134_HGU133P.csv
        GSM1228866_9390_Control_SLE_G6_SPR134_HGU133P.csv
        GSM1228880_6630_Control_SLE_G5_HGU133P.csv
        GSM1228867_9392_Control_SLE_G8_SPR134_HGU133P.csv
        GSM1228881_6633_Control_SLE_M3_HGU133P.csv
        GSM1228879_6629_Control_SLE_G4_HGU133P.csv
        GSM1228860_9391_Control_SLE_G7_SPR134_HGU133P.csv
        GSM1228870_9396_Control_SLE_M7_SPR134_HGU133P.csv
        GSM1228868_9393_Control_SLE_G9_SPR134_HGU133P.csv
        GSM1228917_6631_Control_SLE_M1_HGU133P.csv
        GSM1228878_6627_Control_SLE_G2_HGU133P.csv
        GSM1228862_9397_Control_SLE_M8_SPR134_HGU133P.csv
        GSM1228882_6634_Control_SLE_M4_HGU133P.csv
        GSM1228872_9399_Control_SLE_M10_SPR134_HGU133P.csv
        GSM1228918_6635_Control_SLE_M5_HGU133P.csv
        GSM1228869_9395_Control_SLE_M6_SPR134_HGU133P.csv
        GSM1228876_6632_Control_SLE_M2_HGU133P.csv
 

In [23]:
import os
import shutil

# Define paths
base_path = "/root/.cache/kagglehub/datasets/abinash6000/sle-gene-expression-profiles-7000/versions/1"
control_folder = os.path.join(base_path, "Control")
sle_folder = os.path.join(base_path, "SLE")

# Create directories if not exist
os.makedirs(control_folder, exist_ok=True)
os.makedirs(sle_folder, exist_ok=True)

# Loop through files and move them accordingly
for file_name in os.listdir(base_path):
    file_path = os.path.join(base_path, file_name)

    if os.path.isfile(file_path):  # Ensure it's a file
        if "Control" in file_name:
            shutil.move(file_path, os.path.join(control_folder, file_name))
        else:
            shutil.move(file_path, os.path.join(sle_folder, file_name))

print("Files have been organized into 'Control' and 'SLE' folders.")


Files have been organized into 'Control' and 'SLE' folders.


In [24]:
import os

# Define paths
base_path = "/root/.cache/kagglehub/datasets/abinash6000/sle-gene-expression-profiles-7000/versions/1"
control_folder = os.path.join(base_path, "Control")
sle_folder = os.path.join(base_path, "SLE")

# Count files in each folder
control_count = len([f for f in os.listdir(control_folder) if os.path.isfile(os.path.join(control_folder, f))])
sle_count = len([f for f in os.listdir(sle_folder) if os.path.isfile(os.path.join(sle_folder, f))])

# Print results
print(f"Number of files in 'Control' folder: {control_count}")
print(f"Number of files in 'SLE' folder: {sle_count}")

Number of files in 'Control' folder: 20
Number of files in 'SLE' folder: 61


In [25]:
platform_file="/content/drive/MyDrive/GPL570-55999.txt"
cdf_file="/content/drive/MyDrive/HG-U133_Plus_2.cdf.gz"

In [26]:
import pandas as pd
# Step 1: Open and inspect the file structure
with open(platform_file, 'r', encoding='latin-1') as f:
    lines = f.readlines()

# Step 2: Find the start of actual table data
start_idx = None
for i, line in enumerate(lines):
    if line.startswith("ID\t"):
        start_idx = i
        break

# Step 3: Load the table from the detected line onwards
if start_idx is not None:
    from io import StringIO
    table_data = "".join(lines[start_idx:])
    gpl_df = pd.read_csv(StringIO(table_data), sep="\t", low_memory=False)

    # Filter for relevant columns if present
    columns_needed = ["ID", "Gene Symbol", "Gene Title"]
    gpl_df = gpl_df[[col for col in columns_needed if col in gpl_df.columns]]
    gpl_df.columns = ["probe_id", "gene_symbol", "gene_title"]

    # Preview
    print(gpl_df.head())
else:
    print("Could not find start of table data in GPL file.")


    probe_id       gene_symbol  \
0  1007_s_at  DDR1 /// MIR4640   
1    1053_at              RFC2   
2     117_at             HSPA6   
3     121_at              PAX8   
4  1255_g_at            GUCA1A   

                                          gene_title  
0  discoidin domain receptor tyrosine kinase 1 //...  
1        replication factor C (activator 1) 2, 40kDa  
2               heat shock 70kDa protein 6 (HSP70B')  
3                                       paired box 8  
4            guanylate cyclase activator 1A (retina)  


In [27]:
def extract_mean_features(folder_path):
    import pandas as pd
    import os

    features = []
    for file in sorted(os.listdir(folder_path)):
        file_path = os.path.join(folder_path, file)
        try:
            df = pd.read_csv(file_path)

            # Check for exact column names
            if all(col in df.columns for col in ['X', 'Y', 'MEAN']):
                df['coord'] = df['X'].astype(str) + "_" + df['Y'].astype(str)
                mean_series = df.set_index('coord')['MEAN']
                features.append(mean_series.rename(file))
            else:
                print(f"Skipping file (missing columns): {file}")
        except Exception as e:
            print(f"Error reading {file}: {e}")

    combined = pd.concat(features, axis=1).T
    return combined


In [28]:
X_control = extract_mean_features(control_folder)
X_sle = extract_mean_features(sle_folder)

# Combine and add labels
X_all = pd.concat([X_control, X_sle], axis=1).T
y = [0] * X_control.shape[1] + [1] * X_sle.shape[1]  # 0 = Control, 1 = SLE

In [29]:
import pandas as pd

# Load platform annotation file (manually cleaned GPL570 file)
platform_df = pd.read_csv(platform_file, sep="\t", comment='#', low_memory=False)

# Preview columns
print(platform_df.columns)


Index(['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date',
       'Sequence Type', 'Sequence Source', 'Target Description',
       'Representative Public ID', 'Gene Title', 'Gene Symbol',
       'ENTREZ_GENE_ID', 'RefSeq Transcript ID',
       'Gene Ontology Biological Process', 'Gene Ontology Cellular Component',
       'Gene Ontology Molecular Function'],
      dtype='object')


In [30]:


%load_ext rpy2.ipython



The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [33]:
%%R
# Install affxparser if not already installed
if (!requireNamespace("BiocManager", quietly = TRUE)) {
    install.packages("BiocManager", repos='https://cran.rstudio.com')
}
BiocManager::install("affxparser", force = TRUE)

# Load the library
library(affxparser)

# Path to your CDF file
cdf_file <- "/content/drive/MyDrive/HG-U133_Plus_2.cdf"

# Read the CDF file
cdf_data <- readCdf(cdf_file)

# Show structure
str(cdf_data[[1]])


List of 7
 $ groups       :List of 1
  ..$ AFFX-BioB-5_at:List of 9
  .. ..$ x             : int [1:40] 574 574 575 575 576 576 577 577 578 578 ...
  .. ..$ y             : int [1:40] 682 683 682 683 682 683 682 683 682 683 ...
  .. ..$ pbase         : chr [1:40] "A" "T" "C" "G" ...
  .. ..$ tbase         : chr [1:40] "T" "T" "G" "G" ...
  .. ..$ atom          : int [1:40] 0 0 1 1 2 2 3 3 4 4 ...
  .. ..$ indexpos      : int [1:40] 0 0 1 1 2 2 3 3 4 4 ...
  .. ..$ groupdirection: chr "antisense"
  .. ..$ natoms        : int 20
  .. ..$ ncellsperatom : int 2
 $ unittype     : chr "expression"
 $ unitdirection: chr "antisense"
 $ natoms       : int 20
 $ ncells       : int 40
 $ ncellsperatom: int 2
 $ unitnumber   : int 1


'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com
Bioconductor version 3.20 (BiocManager 1.30.25), R 4.4.3 (2025-02-28)
Installing package(s) 'affxparser'
trying URL 'https://bioconductor.org/packages/3.20/bioc/src/contrib/affxparser_1.78.0.tar.gz'
Content type 'application/x-gzip' length 818689 bytes (799 KB)
downloaded 799 KB


The downloaded source packages are in
	‘/tmp/RtmpzayUHz/downloaded_packages’

Attaching package: ‘affxparser’

The following object is masked from ‘package:R.utils’:

    findFiles



In [38]:
%%R
# Initialize empty list to store results
mapping_list <- list()

# Iterate over all ProbeSet IDs
for (probeset_id in names(cdf_data)) {
  groups <- cdf_data[[probeset_id]]$groups
  for (group_name in names(groups)) {
    group <- groups[[group_name]]
    coords <- data.frame(
      X = group$x,
      Y = group$y,
      ProbeSet_ID = probeset_id
    )
    mapping_list[[length(mapping_list) + 1]] <- coords
  }
}

# Combine all mappings into one data frame
mapping_df <- do.call(rbind, mapping_list)

# Save to CSV
write.csv(mapping_df, "/content/XY_to_Probeset.csv", row.names = FALSE)

# Show a sample
head(mapping_df)


    X   Y    ProbeSet_ID
1 574 682 AFFX-BioB-5_at
2 574 683 AFFX-BioB-5_at
3 575 682 AFFX-BioB-5_at
4 575 683 AFFX-BioB-5_at
5 576 682 AFFX-BioB-5_at
6 576 683 AFFX-BioB-5_at


In [39]:
import os
import pandas as pd

# Paths
xy_mapping_file = "/content/XY_to_Probeset.csv"

# Load XY to ProbeSet mapping
xy_map = pd.read_csv(xy_mapping_file)

# Function to process each file
def process_file(file_path):
    df = pd.read_csv(file_path)
    merged = pd.merge(df, xy_map, on=["X", "Y"], how="inner")
    probeset_mean = merged.groupby("ProbeSet_ID")["MEAN"].mean()
    return probeset_mean

# Prepare list to collect samples
data = []
labels = []
sample_ids = []

# Process control files
for fname in os.listdir(control_folder):
    fpath = os.path.join(control_folder, fname)
    sample_series = process_file(fpath)
    data.append(sample_series)
    labels.append(0)  # Label: Control = 0
    sample_ids.append(f"CONTROL_{fname}")

# Process SLE files
for fname in os.listdir(sle_folder):
    fpath = os.path.join(sle_folder, fname)
    sample_series = process_file(fpath)
    data.append(sample_series)
    labels.append(1)  # Label: SLE = 1
    sample_ids.append(f"SLE_{fname}")

# Combine all into a DataFrame
expression_df = pd.DataFrame(data)
expression_df.index = sample_ids
expression_df["Label"] = labels

# Save to CSV
expression_df.to_csv("/content/samplewise_expression_matrix.csv")
print("Saved sample-wise matrix: /content/samplewise_expression_matrix.csv")


Saved sample-wise matrix: /content/samplewise_expression_matrix.csv


In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Load data
file_path = '/content/samplewise_expression_matrix.csv'  # Update if different
df = pd.read_csv(file_path)

# Rename first column as "Filename"
df.rename(columns={df.columns[0]: "Filename"}, inplace=True)

# Generate labels from filename
df['Label'] = df['Filename'].apply(lambda x: 'Control' if 'control' in x.lower() else 'SLE')

# Drop filename column (we don’t need it as a feature)
df.drop(columns=['Filename'], inplace=True)
df.head()

,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,1431_at,...,AFFX-r2-Ec-bioB-3_at,AFFX-r2-Ec-bioB-5_at,AFFX-r2-Ec-bioB-M_at,AFFX-r2-Ec-bioC-3_at,AFFX-r2-Ec-bioC-5_at,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at,Label
0,354.50000,154.00000,204.28125,349.15625,271.93750,279.87500,166.65625,108.37500,741.035714,95.96875,...,254.818182,279.000000,246.909091,562.636364,420.727273,1981.590909,1885.772727,5791.590909,6479.318182,Control
1,540.93750,222.96875,350.21875,487.50000,595.71875,458.65625,238.28125,156.50000,2228.071429,122.31250,...,363.363636,376.818182,381.363636,755.318182,615.136364,2701.636364,2272.000000,8368.636364,8487.090909,Control
2,639.84375,272.84375,364.37500,602.06250,633.53125,644.28125,290.09375,201.68750,3032.571429,153.56250,...,419.181818,466.500000,462.909091,888.545455,754.909091,3203.227273,2755.727273,9198.500000,9579.954545,Control
3,384.75000,153.37500,294.43750,378.15625,282.59375,390.28125,171.12500,111.71875,1634.321429,90.46875,...,265.227273,297.409091,259.590909,590.045455,438.272727,2112.545455,2010.545455,6149.818182,7121.181818,Control
4,567.03125,245.43750,257.46875,532.59375,386.50000,577.87500,262.31250,167.56250,2578.392857,135.75000,...,368.727273,377.227273,383.772727,717.909091,594.545455,2666.818182,4804.681818,8188.409091,8494.363636,Control


In [51]:
output_path = '/content/drive/MyDrive/expression_data_with_labels.csv'
df.to_csv(output_path, index=False)

In [49]:
# Shuffle the DataFrame rows
df = df.sample(frac=1, random_state=42).reset_index(drop=True)


In [50]:
# Separate features and target
X = df.drop('Label', axis=1)
y = df['Label']

# Encode labels to 0 and 1
le = LabelEncoder()
y_encoded = le.fit_transform(y)  # Control -> 0, SLE -> 1

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded, shuffle=True)

# ---- CLASSIFIER 1: Logistic Regression ----
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)
y_pred_lr = logreg.predict(X_test)

# ---- CLASSIFIER 2: Random Forest ----
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# ---- EVALUATION ----
print("=== Logistic Regression ===")
print(classification_report(y_test, y_pred_lr, target_names=le.classes_))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lr))

print("\n=== Random Forest ===")
print(classification_report(y_test, y_pred_rf, target_names=le.classes_))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))

=== Logistic Regression ===
              precision    recall  f1-score   support

     Control       0.80      1.00      0.89         4
         SLE       1.00      0.92      0.96        13

    accuracy                           0.94        17
   macro avg       0.90      0.96      0.92        17
weighted avg       0.95      0.94      0.94        17

Confusion Matrix:
 [[ 4  0]
 [ 1 12]]

=== Random Forest ===
              precision    recall  f1-score   support

     Control       0.80      1.00      0.89         4
         SLE       1.00      0.92      0.96        13

    accuracy                           0.94        17
   macro avg       0.90      0.96      0.92        17
weighted avg       0.95      0.94      0.94        17

Confusion Matrix:
 [[ 4  0]
 [ 1 12]]
